# ---
# jupyter:
#   jupytext:
#     text_representation:
#       extension: .py
#       format_name: light
#       format_version: '1.5'
#       jupytext_version: 1.16.1
#   kernelspec:
#     display_name: supplychain
#     language: python
#     name: python3
# ---

# %% [markdown]
# # Supply Chain Route Optimization 🚛🌍
# **Author**: Logistics Optimization Team  
# **Last Updated**: 2023-11-20  
# **Version**: 1.2.0

# %% [markdown]
# ## 1. Environment Setup

# %%
# Core

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from datetime import datetime


# Geospatial

In [ ]:
import osmnx as ox
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster


# Optimization

In [ ]:
from route_optimizer.graphhopper_api import GraphHopperClient
from route_optimizer.carbon_footprint import CarbonCalculator


# Blockchain

In [ ]:
from blockchain.web3_integration import BlockchainIntegrator

In [ ]:
# Utilities

In [ ]:
import json
from tqdm import tqdm
warnings.filterwarnings('ignore')

# %%
# Configuration

In [ ]:
ox.config(log_console=True, use_cache=True)
GRAPHHOPPER_KEY = os.getenv("GRAPHHOPPER_API_KEY")
CONTRACT_ADDRESS = "0x123...supplychain-contract-address"


# %% [markdown]
# ## 2. Transportation Network Analysis


# %%

In [ ]:
def visualize_network(location="Berlin, Germany", network_type="drive"):
    """Visualize city transportation network"""
    G = ox.graph_from_place(location, network_type=network_type)
    fig, ax = ox.plot_graph(G, node_size=0, edge_linewidth=0.5)
    
    # Calculate network stats
    stats = ox.basic_stats(G)
    print(f"Network Density: {stats['street_density_km']:.2f} km/km²")
    print(f"Total Road Length: {stats['street_length_total']/1000:.1f} km")
    
    return G

# %%
# Generate Berlin road network

In [ ]:
G = visualize_network()

# %% [markdown]
# ## 3. Single Route Optimization

# %%

In [ ]:
def optimize_single_route(origin, destination, vehicle_type="truck"):
    """Optimize route between two points"""
    gh = GraphHopperClient()
    
    # Get coordinates
    origin_geo = ox.geocode(origin)
    dest_geo = ox.geocode(destination)
    
    # Optimize route
    route = gh.get_route(
        RouteOptimizationRequest(
            waypoints=[origin_geo, dest_geo],
            vehicle=vehicle_type
        )
    )
    
    # Calculate emissions
    calc = CarbonCalculator()
    emissions = calc.calculate_co2(
        route.distance/1000, 
        vehicle_type="diesel_truck",
        load_kg=5000
    )
    
    return route, emissions


# %%
# Optimize Berlin to Hamburg route

In [ ]:
route, emissions = optimize_single_route(
    "Berlin Central Station",
    "Hamburg Port",
    vehicle_type="heavy_truck"
)

print(f"Optimal Route: {route.distance/1000:.1f} km")
print(f"Estimated Emissions: {emissions} kg CO₂")

In [ ]:
def optimize_multi_stop(waypoints, constraints):
    """Optimize route with multiple stops"""
    gh = GraphHopperClient()
    
    # Get coordinates
    points = [ox.geocode(w) for w in waypoints]
    
    # Get distance matrix
    matrix = gh.calculate_distance_matrix(points)
    
    # Solve TSP
    optimal_order = solve_tsp(matrix['durations'])
    
    # Build final route
    optimized_route = []
    for i in range(len(optimal_order)-1):
        origin = points[optimal_order[i]]
        dest = points[optimal_order[i+1]]
        optimized_route += gh.get_route(
            RouteOptimizationRequest([origin, dest])
        )
    
    return optimized_route

In [ ]:
waypoints = [
    "Berlin Warehouse",
    "Potsdam DC",
    "Leipzig Factory",
    "Dresden Retail"
]
optimized_route = optimize_multi_stop(waypoints, constraints={
    'max_duration': 8*3600,
    'vehicle_capacity': 8000
})

# ## 5. Carbon-Aware Routing


In [ ]:
def compare_routes(origin, destination):
    """Compare different transport modes"""
    modes = ['truck', 'train', 'ship']
    results = []
    
    for mode in modes:
        route = optimize_single_route(origin, destination, mode)
        emissions = CarbonCalculator().calculate_co2(
            route.distance/1000, 
            vehicle_type=mode,
            load_kg=5000
        )
        results.append({
            'mode': mode,
            'distance': route.distance,
            'emissions': emissions,
            'cost': estimate_cost(route, mode)
        })
    
    return pd.DataFrame(results)

In [ ]:
comparison = compare_routes("Frankfurt", "Milan")
comparison.style.background_gradient(cmap='viridis')

In [ ]:
def log_route_to_blockchain(route_details):
    """Record optimized route on blockchain"""
    integrator = BlockchainIntegrator()
    contract = integrator.get_contract()
    
    tx_receipt = contract.functions.logRoute(
        route_details['hash'],
        int(route_details['distance']),
        int(route_details['emissions']*1000),  # Convert to grams
        int(datetime.now().timestamp())
    ).transact({
        'gas': 200000,
        'gasPrice': integrator.w3.toWei('50', 'gwei')
    })
    
    return tx_receipt

# Example blockchain logging

In [ ]:
route_details = {
    'hash': '0x123...optimized-route-hash',
    'distance': optimized_route.distance,
    'emissions': emissions
}
receipt = log_route_to_blockchain(route_details)
print(f"Blockchain TX Hash: {receipt.transactionHash.hex()}")


In [ ]:
def plot_optimized_route(route):
    """Create interactive route map"""
    m = folium.Map(location=route[0].geometry.coords[0][::-1], zoom_start=8)
    
    # Add route segments
    for segment in route:
        folium.PolyLine(
            locations=[(lat, lon) for lon, lat in segment.geometry.coords],
            color='#FF0000',
            weight=3
        ).add_to(m)
    
    # Add markers
    MarkerCluster().add_to(m)
    for idx, point in enumerate(route.waypoints):
        folium.Marker(
            location=point[::-1],
            popup=f"Stop {idx+1}",
            icon=folium.Icon(color='green' if idx==0 else 'blue')
        ).add_to(m)
        
    return m

# Display interactive map